<a href="https://colab.research.google.com/github/Praven-techdeveloper/txt-to-img_AI/blob/main/Txt2img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet \
  diffusers transformers accelerate safetensors \
  huggingface_hub ipywidgets torch Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.4 MB/s eta 0:00:00


In [ ]:
import torch
from functools import lru_cache
from IPython.display import display
from huggingface_hub import login as hf_login
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from diffusers.schedulers import (
    DDIMScheduler,
    DPMSolverMultistepScheduler,
    LMSDiscreteScheduler
)
import ipywidgets as widgets

In [ ]:
# Custom scheduler example (tweaks DDIM's betas)
class CustomScheduler(DDIMScheduler):
    def __init__(self, config):
        super().__init__(config)
        self.betas = self.betas * 0.8

    @classmethod
    def from_config(cls, config):
        return cls(config)

# Map names to classes
SCHED_MAP = {
    "DDIM": DDIMScheduler,
    "DPMSolverMultistep": DPMSolverMultistepScheduler,
    "LMSDiscrete": LMSDiscreteScheduler,
    "Custom": CustomScheduler,
}

def get_scheduler(name, config):
    cls = SCHED_MAP.get(name)
    if cls is None:
        raise ValueError(f"Unknown scheduler: {name}")
    return cls.from_config(config)

In [ ]:
@lru_cache(maxsize=None)
def load_sd15(token: str):
    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16,
        use_auth_token=token or True
    )
    return pipe.to("cuda")

@lru_cache(maxsize=None)
def load_sdxl(token: str):
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        use_auth_token=token or True
    )
    return pipe.to("cuda")

In [ ]:
# 5.1 Hugging Face Token
token_widget = widgets.Password(
    description="HF Token",
    placeholder="Paste your token…",
    layout=widgets.Layout(width="400px")
)

def on_token_change(change):
    hf_login(token_widget.value, add_to_git_credential=False)

token_widget.observe(on_token_change, names="value")

# 5.2 Model Selection
model_widget = widgets.Dropdown(
    options=["Stable Diffusion 1.5", "Stable Diffusion XL"],
    description="Model:"
)

# 5.3 Scheduler Selection
sched_widget = widgets.Dropdown(
    options=list(SCHED_MAP.keys()),
    description="Scheduler:"
)

# 5.4 Prompt Input & Generate Button
prompt_widget = widgets.Text(
    description="Prompt:",
    placeholder="e.g. A serene castle floating in clouds",
    layout=widgets.Layout(width="600px")
)
generate_btn = widgets.Button(description="🚀 Generate", button_style="primary")

# Display all controls
ui = widgets.VBox([
    token_widget,
    widgets.HBox([model_widget, sched_widget]),
    prompt_widget,
    generate_btn
])
display(ui)

In [ ]:
out = widgets.Output()
display(out)

def run_generation(_):
    out.clear_output()
    prompt = prompt_widget.value.strip()
    token  = token_widget.value.strip()
    model  = model_widget.value
    sched  = sched_widget.value

    if not token:
        with out:
            print("⚠️ Please enter your Hugging Face token first.")
        return
    if not prompt:
        with out:
            print("⚠️ Please enter a prompt.")
        return

    # Load pipeline
    if model == "Stable Diffusion 1.5":
        pipe = load_sd15(token)
    else:
        pipe = load_sdxl(token)

    # Swap scheduler
    pipe.scheduler = get_scheduler(sched, pipe.scheduler.config)

    # Fix seed for reproducibility
    torch.manual_seed(42)

    # Generate
    image = pipe(prompt).images[0]

    # Display result
    with out:
        display(image)

generate_btn.on_click(run_generation)

Output()